In [5]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir) 

In [6]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, funksvd, user_knn as uknn, item_knn as iknn 
from lenskit import topn, topnFair

In [8]:
import numpy as np
import pandas as pd
import math
%matplotlib inline

In [9]:
#dataset = os.path.join(parentdir,'ml-latest-small/ratings.1.csv')    
#ratings = pd.read_csv(dataset, sep=',',
#                      names=['user', 'item', 'rating', 'timestamp'], header=0)

dataset = os.path.join(parentdir,'movielens100k/u.data')     
ratings = pd.read_csv(dataset, sep='\s+',
                      names=['user', 'item', 'rating', 'timestamp'], header=0)

items = pd.read_csv('/Users/josse/Git-kode projects/Speciale/Fair-Recommendations/ml-latest-small/movies.csv', sep=',',
                      names=['item','title','genres'], header=0)

In [10]:
items_dummy = pd.concat([items, items['genres'].str.get_dummies(sep='|')], axis=1)
items_dummy.head()

,item,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
items_n = ratings.item.nunique()
items_n

1682

In [11]:
algo_uu = uknn.UserUser(20)
algo_ii = iknn.ItemItem(20)
algo_als = als.BiasedMF(50)
algo_funkSVD = funksvd.FunkSVD(50)


In [36]:
def eval(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    
    # now we run the recommender
    recs = batch.recommend(fittable, users, items_n)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [37]:
all_recs = []
test_data = []
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
    test_data.append(test)
    all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs.append(eval('UserUser', algo_uu, train, test))
    all_recs.append(eval('ALS', algo_als, train, test))
    all_recs.append(eval('FunkSVD', algo_funkSVD, train, test))
    

In [38]:
test_data = pd.concat(test_data, ignore_index=True)
all_recs = pd.concat(all_recs, ignore_index=True)
all_recs_joined = all_recs.join(items_dummy.set_index('item'), on='item')
all_recs_joined.head()

,item,score,user,rank,Algorithm,title,genres,(no genres listed),Action,Adventure,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1449,4.935732,2,1,ItemItem,Waiting for Guffman (1996),Comedy,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1594,4.828795,2,2,ItemItem,In the Company of Men (1997),Comedy|Drama,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,483,4.729869,2,3,ItemItem,King of the Hill (1993),Drama,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,169,4.718284,2,4,ItemItem,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1191,4.717778,2,5,ItemItem,Madonna: Truth or Dare (1991),Documentary|Musical,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
#test = all_recs_joined.loc[all_recs_joined["user"]==1]
#test = test.loc[test["Algorithm"] == "ALS"]
#test.reset_index()
#test.iloc[90:100,:]

In [39]:
#providers = list(all_recs_joined.iloc[:,8:])
#items_n = all_recs_joined.item.nunique()
#proItems_n =  all_recs_joined.loc[ all_recs_joined['Drama'] == 1].item.nunique()
#proItems_n_horror =  all_recs_joined.loc[ all_recs_joined['Horror'] == 1].item.nunique()

In [17]:
#rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
#rla.add_metric("rND")
#rla.add_metric("rKL")
#rla.add_metric("rRD")
#rla.add_metric("div")
#rla.add_metric("dem_parity")
#rla.add_metric("ndcg")
#results = rla.compute(all_recs_joined, test_data, "Drama", providers, items_n, proItems_n)
#results.head()

C:\Users\josse\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


rND       rKL       rRD       div  dem_parity      ndcg
user Algorithm                                                              
1    ALS        0.134752  0.056526  0.104964  0.894737    0.757924  0.224042
     FunkSVD    0.054564  0.006826  0.138085  0.947368    0.599851  0.095000
     ItemItem   0.060650  0.012008  0.089514  0.894737    0.640199  0.222759
     UserUser   0.096737  0.023389  0.233847  0.894737    0.431029  0.158302
2    ALS        0.070125  0.017061  0.024953  0.842105    0.749600  0.180893

In [18]:
#results.groupby('Algorithm').mean()

,rND,rKL,rRD,div,dem_parity,ndcg
Algorithm,,,,,,
ALS,0.116014,0.044618,0.075972,0.911871,0.733497,0.132058
FunkSVD,0.088580,0.027650,0.088439,0.934587,0.700787,0.083109
ItemItem,0.110265,0.038038,0.082345,0.910030,0.707331,0.096400
UserUser,0.138217,0.062169,0.287409,0.926662,0.541222,0.064084


In [19]:
#rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
#rla.add_metric("rND")
#rla.add_metric("rKL")
#rla.add_metric("rRD")
#rla.add_metric("div")
#rla.add_metric("dem_parity")
#rla.add_metric("ndcg")
#results_horror = rla.compute(all_recs_joined, test_data, "Horror", providers, items_n, proItems_n_horror)
#results_horror.groupby('Algorithm').mean()

,rND,rKL,rRD,div,dem_parity,ndcg
Algorithm,,,,,,
ALS,0.049296,0.038934,0.024357,0.911871,0.048216,0.132058
FunkSVD,0.036278,0.026602,0.024253,0.934587,0.047200,0.083109
ItemItem,0.047376,0.041115,0.038100,0.910030,0.037880,0.096400
UserUser,0.049524,0.045453,0.040889,0.926662,0.043750,0.064084


In [20]:
#allResults = []
#allResults.append(results.groupby('Algorithm').mean())
#allResults.append(results_horror.groupby('Algorithm').mean())

In [21]:
#pd.concat(allResults, keys=['Drama', 'Horror'])

rND       rKL       rRD       div  dem_parity      ndcg
       Algorithm                                                              
Drama  ALS        0.116014  0.044618  0.075972  0.911871    0.733497  0.132058
       FunkSVD    0.088580  0.027650  0.088439  0.934587    0.700787  0.083109
       ItemItem   0.110265  0.038038  0.082345  0.910030    0.707331  0.096400
       UserUser   0.138217  0.062169  0.287409  0.926662    0.541222  0.064084
Horror ALS        0.049296  0.038934  0.024357  0.911871    0.048216  0.132058
       FunkSVD    0.036278  0.026602  0.024253  0.934587    0.047200  0.083109
       ItemItem   0.047376  0.041115  0.038100  0.910030    0.037880  0.096400
       UserUser   0.049524  0.045453  0.040889  0.926662    0.043750  0.064084

In [29]:
def evaluate_genre(genre): 
    
    proItems_genre_n =  all_recs_joined.loc[ all_recs_joined[genre] == 1].item.nunique()
    print (genre)
    rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
    rla.add_metric("rND")
    rla.add_metric("rKL")
    rla.add_metric("rRD")
    rla.add_metric("div")
    rla.add_metric("dem_parity")
    rla.add_metric("ndcg")
    results= rla.compute(all_recs_joined, test_data, genre, providers, items_n, proItems_genre_n)
    #results_horror.groupby('Algorithm').mean()
    print("done" , genre)
    return results.groupby('Algorithm').mean()

In [40]:
testproviders = ["Drama", "Action"]
allResults = []
for genre in testproviders:
    rep = evaluate_genre(genre)
    allResults.append(rep)
pd.concat(allResults, keys=testproviders)
    

1
2
3
4
5
100
1
2
3
4
5
100


rND       rKL       rRD  div  dem_parity      ndcg
       Algorithm                                                         
Drama  ALS        0.288459  0.055931  0.395489  1.0    0.660174  0.321699
       FunkSVD    0.280887  0.037913  0.535419  1.0    0.659872  0.284259
       ItemItem   0.256824  0.053717  0.194461  1.0    0.675981  0.298044
       UserUser   0.310423  0.069552  0.671488  1.0    0.651334  0.270667
Action ALS        0.281926  0.087632  0.041376  1.0    0.151768  0.321699
       FunkSVD    0.327208  0.117836  0.050031  1.0    0.150894  0.284259
       ItemItem   0.229536  0.073896  0.078630  1.0    0.149654  0.298044
       UserUser   0.259622  0.063289  0.065402  1.0    0.149787  0.270667